# Analysis LSR vs Lexical Efficiency

We run a set of retrieval engines on both, lexical (I.e., BM25 weights) and learned sparse embeddings to calculate how much, on average the efficiency gap between learned sparse and lexical is.

In [1]:
import pandas as pd

df = pd.read_json("lsr-vs-lexical.jsonl.gz", lines=True)

In [2]:
df[["Retrieval", "lexical", "retrieval_per_query.runtime_wallclock", "nDCG@10"]].groupby(["Retrieval", "lexical"]).describe(percentiles=[0.5, .9, .99])

retrieval_per_query.runtime_wallclock  \
                                                              count   
Retrieval             lexical                                         
duckdb                False                                   180.0   
                      True                                     15.0   
naive-search          False                                   180.0   
                      True                                     15.0   
pyterrier-splade      False                                   180.0   
                      True                                     15.0   
pyterrier-splade-pisa False                                   180.0   
                      True                                     15.0   
seismic               False                                   180.0   
                      True                                     15.0   

                                                                           \
                                    mean        std        min        50%   
Retrieval             lexical                                               
duckdb                False    32.212080  47.097088   4.680000  12.350000   
                      True      6.706895   1.308715   4.465116   6.755102   
naive-search          False    22.428912  20.867778   2.139535   9.570000   
                      True      3.720603   1.659243   1.116279   3.300000   
pyterrier-splade      False    31.473835  15.977443  15.160000  24.790913   
                      True     20.964726   3.990952  15.061224  20.259259   
pyterrier-splade-pisa False    10.330056  16.349951   0.740000   3.270000   
                      True      0.832182   0.273555   0.469388   0.760000   
seismic               False     1.256967   1.131683   0.139535   0.700000   
                      True      0.037264   0.009419   0.020000   0.040000   

                                                                  nDCG@10  \
                                      90%         99%         max   count   
Retrieval             lexical                                               
duckdb                False    108.568000  188.717013  274.730337   180.0   
                      True       8.344000    8.514800    8.540000    15.0   
naive-search          False     54.820365   73.512842   94.382022   180.0   
                      True       5.264000    7.576485    7.943820    15.0   
pyterrier-splade      False     51.596790   89.535169  124.168539   180.0   
                      True      26.324000   26.818017   26.895833    15.0   
pyterrier-splade-pisa False     31.341417   69.464607  132.550562   180.0   
                      True       1.068000    1.570387    1.640449    15.0   
seismic               False      2.945438    4.308000    5.780000   180.0   
                      True       0.041327    0.054148    0.056180    15.0   

                                                                       \
                                   mean       std       min       50%   
Retrieval             lexical                                           
duckdb                False    0.385041  0.171082  0.074498  0.399728   
                      True     0.265785  0.173558  0.015273  0.342109   
naive-search          False    0.384960  0.171117  0.073129  0.399927   
                      True     0.265531  0.172538  0.017147  0.341152   
pyterrier-splade      False    0.384836  0.170948  0.072610  0.398859   
                      True     0.266480  0.173446  0.015531  0.340266   
pyterrier-splade-pisa False    0.384872  0.170988  0.072610  0.398854   
                      True     0.266326  0.173165  0.015531  0.340902   
seismic               False    0.385097  0.170966  0.073129  0.398987   
                      True     0.215713  0.095977  0.079309  0.223830   

                                                             
                                    90%       99%       max  
Retrieval             lexica

In [ ]:
def fill(f, cnt):
    for _ in range(cnt-len(f)):
        f = '\\phantom{0}' + f
    return f

def line(v):
    df_eval = df[["Retrieval", "lexical", "retrieval_per_query.runtime_wallclock", "nDCG@10"]].groupby(["Retrieval", "lexical"]).describe(percentiles=[0.5, .9, .99]).reset_index()
    df_eval = df_eval[df_eval["Retrieval"] == v]
    ret = []
    for i in [False, True]:
        x = df_eval[df_eval["lexical"] == i]
        if len(x) != 1:
            raise ValueError("Expected exactly one row.")
        x = x.to_dict()
        ret += [
            fill("{:.3f}".format(list(x[('retrieval_per_query.runtime_wallclock', '50%')].values())[0]), 6),
            fill("{:.3f}".format(list(x[('retrieval_per_query.runtime_wallclock', '90%')].values())[0]), 8),
            fill("{:.3f}".format(list(x[('retrieval_per_query.runtime_wallclock', '99%')].values())[0]), 8),
            fill("{:.3f}".format(list(x[('nDCG@10', 'mean')].values())[0]), 4),
        ]
    return " & ".join(ret)

for k, v in [("DuckDB", "duckdb"), ("Na{\\\"i}ve", "naive-search"), ("PISA", "pyterrier-splade-pisa"), ("PyTerrier", "pyterrier-splade"), ("Seismic", "seismic")]:
    print(k,"&", line(v), "\\\\")

DuckDB & 12.350 & \phantom{0}108.568 & \phantom{0}188.717 & 0.385 & \phantom{0}6.755 & \phantom{0}\phantom{0}\phantom{0}8.344 & \phantom{0}\phantom{0}\phantom{0}8.515 & 0.266 \\
Na{\"i}ve & \phantom{0}9.570 & \phantom{0}\phantom{0}54.820 & \phantom{0}\phantom{0}73.513 & 0.385 & \phantom{0}3.300 & \phantom{0}\phantom{0}\phantom{0}5.264 & \phantom{0}\phantom{0}\phantom{0}7.576 & 0.266 \\
PISA & \phantom{0}3.270 & \phantom{0}\phantom{0}31.341 & \phantom{0}\phantom{0}69.465 & 0.385 & \phantom{0}0.760 & \phantom{0}\phantom{0}\phantom{0}1.068 & \phantom{0}\phantom{0}\phantom{0}1.570 & 0.266 \\
PyTerrier & 24.791 & \phantom{0}\phantom{0}51.597 & \phantom{0}\phantom{0}89.535 & 0.385 & 20.259 & \phantom{0}\phantom{0}26.324 & \phantom{0}\phantom{0}26.818 & 0.266 \\
Seismic & \phantom{0}0.700 & \phantom{0}\phantom{0}\phantom{0}2.945 & \phantom{0}\phantom{0}\phantom{0}4.308 & 0.385 & \phantom{0}0.040 & \phantom{0}\phantom{0}\phantom{0}0.041 & \phantom{0}\phantom{0}\phantom{0}0.054 & 0.216 \\


# Generate Evaluation DataFrame

The file `lsr-vs-lexical-evaluation.jsonl.gz` used below was created with

```
lsr-benchmark evaluate './*/*/seismic' './*/*/pyterrier-splade-pisa' './*/*/pyterrier-splade' -o lsr-vs-lexical-evaluation.jsonl.gz
```

The results are intermediate, and these steps here slightly enrich the file `lsr-vs-lexical.jsonl.gz` and remove datasets that do not yet have all approaches executed.

In [1]:
import pandas as pd
from lsr_benchmark.datasets import all_embeddings
from tira.rest_api_client import Client
import json

df = pd.read_json('../runs/lsr-vs-lexical-evaluation.jsonl.gz', lines=True)
df["Retrieval"] = df["approach"].apply(lambda i: i.split("/")[-1])
del df["approach"]
APPROACHES = ["pyterrier-splade", "pyterrier-splade-pisa", "seismic", "duckdb", "naive-search"]
tira = Client()
d_stats = {}

for d, d_stat in json.loads(open('../lsr_benchmark/datasets/overview.json').read()).items():
    d_stats[d] = d_stat['dataset_stats']

In [2]:
def normalize_index_runtime(i):
    f = (d_stats[i["tira-dataset-id"]]["docs_count"]/1000)
    if f < 5:
        return None
    return int(i['index.runtime_wallclock'].split(' ')[0]) / f

def normalize_query_runtime(i):
    return int(i['retrieval.runtime_wallclock'].split(' ')[0]) / d_stats[i["tira-dataset-id"]]["queries_count"]

df['index_1000.runtime_wallclock'] = df.apply(normalize_index_runtime, axis=1)
df['retrieval_per_query.runtime_wallclock'] = df.apply(normalize_query_runtime, axis=1)

In [3]:
complete_datasets = set()

print(f"df complete: {len(df)}")

for dataset in df["ir-dataset-id"].unique():
    complete = True
    for approach in APPROACHES:
        if approach in ('pyterrier-naive', 'pyterrier-pisa') and len(df[(df["ir-dataset-id"] == dataset) & (df["Retrieval"] == approach)]) > 0:
            continue

        available_embeddings = df[(df["ir-dataset-id"] == dataset) & (df["Retrieval"] == approach)]["embedding/model"].unique()
        for e in all_embeddings():
            if e not in available_embeddings:
                complete = False
    if complete:
        complete_datasets.add(dataset)

df = df[df["ir-dataset-id"].isin(complete_datasets)]
print(f"df filtered: {len(df)}")

df["lexical"] = df["embedding/model"].map(lambda i: i == "bm25")
df = df[df["Retrieval"].isin(APPROACHES)]
df.to_json('lsr-vs-lexical.jsonl.gz', lines=True, orient="records")

df complete: 1538
df filtered: 1434
